In [ ]:
import os
import random
import logging
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip
from moviepy.audio.fx.all import audio_loop
from moviepy.video.fx.speedx import speedx

# Configure logging
logging.basicConfig(level=logging.WARNING, format='%(levelname)s: %(message)s')

def get_random_file(directory: str, extension: str) -> str:
    """
    Return a random file with the given extension from the specified directory.
    """
    files = [f for f in os.listdir(directory) if f.lower().endswith(extension)]
    if not files:
        raise FileNotFoundError(f"No file with extension {extension} found in {directory}")
    return os.path.join(directory, random.choice(files))

def adjust_audio_speed(audio_clip, target_duration):
    """
    Adjust the audio clip's speed so its duration becomes exactly target_duration.
    This function calculates the necessary speed factor (without limiting it)
    and then trims or loops the adjusted audio if needed.
    """
    current_duration = audio_clip.duration
    if current_duration == target_duration:
        return audio_clip

    # Calculate speed factor; speeding up if narration is longer than needed,
    # or slowing down if it's too short.
    speed_factor = current_duration / target_duration
    adjusted = audio_clip.fx(speedx, speed_factor)

    # Ensure the adjusted audio exactly matches target_duration
    if adjusted.duration < target_duration:
        adjusted = audio_loop(adjusted, duration=target_duration)
    elif adjusted.duration > target_duration:
        adjusted = adjusted.subclip(0, target_duration)
    return adjusted

def assemble_video(i: int):
    """
    Assemble a final video using only:
      - a middle video (from Storage/final_videos/ as book_{i}.mp4)
    It overlays:
      - Background music (from Storage/music_library/song_{i}.mp3) at reduced volume
      - Narration (from Storage/temp_audios/voice_{i}.wav) adjusted to match video duration
    The final output is saved in Video/final_videos/ as video_{i}.mp4.
    """
    # Define directories
    dirs = {
        "middle_videos": "Storage/final_videos/",
        "narrations": "Storage/temp_audios/",
        "music": "Storage/music_library/",
        "final_videos": "Video/final_videos/"
    }

    # Ensure the output directory exists
    os.makedirs(dirs["final_videos"], exist_ok=True)

    try:
        # Construct file paths
        middle_video_path = os.path.join(dirs["middle_videos"], f"book_{i}.mp4")
        narration_path = os.path.join(dirs["narrations"], f"voice_{i}.wav")
        song_path = os.path.join(dirs["music"], f"song_{i}.mp3")

        # Load the video clip
        middle_clip = VideoFileClip(middle_video_path)

        # Calculate duration
        video_duration = middle_clip.duration

        # Load audio clips
        narration_audio = AudioFileClip(narration_path)
        song_audio = AudioFileClip(song_path)

        # Adjust narration speed to exactly match the video duration
        adjusted_narration = adjust_audio_speed(narration_audio, video_duration)

        # Process background music: loop it for the video duration and reduce volume
        music_audio = audio_loop(song_audio, duration=video_duration).volumex(0.2)

        # Combine the narration and background music
        composite_audio = CompositeAudioClip([music_audio, adjusted_narration])

        # Set the new audio track (discarding any native audio from the clip)
        final_clip = middle_clip.set_audio(composite_audio)
        final_video_path = os.path.join(dirs["final_videos"], f"video_{i}.mp4")

        # Write the final video file to disk
        final_clip.write_videofile(
            final_video_path,
            codec="libx264",
            audio_codec="aac",
            temp_audiofile="temp-audio.m4a",
            remove_temp=True,
            verbose=False
        )
        logging.info(f"Video saved to {final_video_path}")

        # Close resources
        middle_clip.close()
        narration_audio.close()
        song_audio.close()
        final_clip.close()

    except Exception as e:
        logging.error(f"An error occurred during video assembly for book {i}: {e}")

def main():
    """
    Process videos for 3 books, handling any potential errors for individual books.
    """
    # Ensure directories exist
    os.makedirs("Storage/final_videos", exist_ok=True)
    os.makedirs("Storage/temp_audios", exist_ok=True)
    os.makedirs("Storage/music_library", exist_ok=True)
    os.makedirs("Video/final_videos", exist_ok=True)

    # Process 3 books
    for book_number in range(1, 4):
        try:
            logging.info(f"Processing video for book {book_number}")
            assemble_video(book_number)
        except Exception as e:
            logging.error(f"Failed to process video for book {book_number}: {e}")

if __name__ == '__main__':
    main()